In [3]:
!pip3 install strands-agents-tools strands-agents anthropic

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 8.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 9.1 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 10.8 MB/s eta 0:00:00 0:00:01m
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 10.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.24.0
    Uninstalling uvicorn-0.24.0:
      Successfully uninstalled uvicorn-0.24.0
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.6
    Uninstalling python-multipart-0.0.6:
      Successfully uninstalled python-m

In [8]:
pip install 'strands-agents[litellm]' strands-agents-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.8/810.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.7/996.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.2 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import json
import requests
import time
import asyncio
from datetime import datetime
from strands import Agent, tool
from strands_tools import calculator
from strands.models.anthropic import AnthropicModel
from strands.models.litellm import LiteLLMModel
import anthropic

import httpx
from datetime import datetime
from typing import Dict, Any

# Configuration - set your API base URL here
BASE_URL = "http://localhost:8000"  # Change this to your deployed URL as needed

@tool
async def get_recent_financial_data(limit: int = 1000) -> dict:
    """
    Get the most recent financial data rows for analysis.
    
    Args:
        limit (int): Number of recent rows to fetch (default: 100)
    
    Returns:
        dict: Financial data response
    """
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{BASE_URL}/api/agent-query",
                params={"limit": limit}
            )
            
            # Check if request was successful
            response.raise_for_status()
            
            # Get the raw data from the agent-query endpoint
            financial_data = response.json()
            
            # Return in the expected format
            return {
                "status": "success",
                "timestamp": datetime.now().isoformat(),
                "data": financial_data,
                "count": len(financial_data),
                "message": f"Retrieved {len(financial_data)} recent financial records for analysis"
            }
    
    except httpx.HTTPError as e:
        # Handle HTTP errors
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "data": [],
            "count": 0,
            "message": f"HTTP error occurred: {str(e)}"
        }
    
    except Exception as e:
        # Handle other errors
        return {
            "status": "error",
            "timestamp": datetime.now().isoformat(),
            "data": [],
            "count": 0,
            "message": f"Error retrieving financial data: {str(e)}"
        }

@tool
def get_current_graphs() -> dict:
    """
    Get all currently displayed graphs.
    
    Returns:
        dict: Response containing list of current graphs
    """
    return {
        "status": "success",
        "timestamp": datetime.now().isoformat(),
        "graphs": [],
        "count": 0,
        "message": "Retrieved current graph portfolio"
    }

@tool
def analyze_and_update_graphs(current_graphs: list) -> dict:
    """
    Analyze current graphs and update them if needed for better trend visualization.
    
    Args:
        current_graphs (list): List of current graph objects
    
    Returns:
        dict: Analysis results and any updates made
    """
    return {
        "status": "success",
        "timestamp": datetime.now().isoformat(),
        "updates_made": 0,
        "updated_graphs": [],
        "analysis": "Analyzed existing graphs for optimization opportunities",
        "recommendations": []
    }

# 1. 
# @tool  
# def find_interesting_relations(financial_data: list) -> dict:
#     """
#     Analyze financial data to find interesting relations between columns using advanced financial analysis techniques.
    
#     Args:
#         financial_data (list): List of financial data records
    
#     Returns:
#         dict: Interesting relations found with financial significance
#     """
    
#     # Turn data into string 
#     data_to_str = str(financial_data)

#     # Ask the model to analyze it 
#     prompt = f"""
#     You are a financial data analyst. Analyze the following financial dataset and
#     extract interesting relations (fraud, profitability, risk, correlations, trends).
        
#     Data sample:
#     {data_str}

#     Respond in JSON with this structure:
#     {{
#       "relations_found": <int>,
#       "relations": [
#         {{
#           "type": "<analysis_type>",
#           "columns": ["col1", "col2"],
#           "strength": "<low|medium|high|numeric>",
#           "insight": "<short summary>"
#         }}
#        ]
#     }}
#     """
    
#     # Run the model 
#     result = model(prompt)

#     # Try parsing 
#     if isinstance(result, str):
#         import json
#         result_json = json.loads(result); 
#     else:
#         result_json = result 
    
#     return {
#         "status": "success",
#         "timestamp": datetime.now().isoformat(),
#         "relations_found": 0,
#         "relations": [],
#         "analysis_types_performed": [
#             "fraud_detection_patterns",
#             "risk_concentration_analysis", 
#             "profitability_trends",
#             "customer_segmentation",
#             "correlation_analysis"
#         ]
#     }

#2. 
# @tool
# def determine_graph_config(relation: dict) -> dict:
#     """
#     Determine the optimal graph type and SQL query for a given financial relation.
    
#     Args:
#         relation (dict): Relation object from find_interesting_relations
    
#     Returns:
#         dict: Optimized graph configuration
#     """
#     return {
#         "status": "success",
#         "graph_type": "bar",
#         "title": "Financial Insight Visualization",
#         "sql_query": "SELECT category, value FROM financial_data GROUP BY category",
#         "description": "Optimized visualization configuration",
#         "rationale": "Graph type selected based on data characteristics and visualization best practices"
#     }

# @tool
# def find_least_useful_graph(current_graphs: list) -> dict:
#     """
#     Analyze current graphs to identify the least useful one for removal based on relevance and value.
    
#     Args:
#         current_graphs (list): List of current graph objects
    
#     Returns:
#         dict: Analysis of least useful graph with detailed reasoning
#     """
#     return {
#         "status": "success",
#         "timestamp": datetime.now().isoformat(),
#         "least_useful_id": None,
#         "reason": "No graphs identified for removal",
#         "usefulness_scores": {},
#         "analysis_criteria": [
#             "data_freshness",
#             "information_value", 
#             "redundancy_check",
#             "statistical_significance"
#         ]
#     }

@tool
def remove_graph(graph_id: int) -> dict:
    """
    Remove a graph by its ID from the dashboard.
    
    Args:
        graph_id (int): ID of the graph to remove
    
    Returns:
        dict: Removal result with confirmation
    """
    return {
        "status": "success",
        "timestamp": datetime.now().isoformat(),
        "removed_id": graph_id,
        "message": f"Successfully removed graph {graph_id} from dashboard"
    }

@tool
def add_new_graph(graph_config: dict) -> dict:
    """
    Add a new graph to the financial dashboard.
    
    Args:
        graph_config (dict): Graph configuration with type, title, sql_query
    
    Returns:
        dict: Addition result with new graph details
    """
    new_id = int(time.time() * 1000) % 100000  # Simple ID generation
    return {
        "status": "success",
        "timestamp": datetime.now().isoformat(), 
        "new_graph_id": new_id,
        "graph": {
            "id": new_id,
            **graph_config
        },
        "message": f"Successfully added new graph: {graph_config.get('title', 'Untitled')}"
    }

@tool
def update_graph(graph_id: int, graph_type: str, sql_query: str, title: str = None) -> dict:
    """
    Update an existing graph's configuration for better visualization.
    
    Args:
        graph_id (int): ID of graph to update
        graph_type (str): New graph type (bar, line, scatter, pie)
        sql_query (str): New SQL query for the graph
        title (str): Optional new title for the graph
    
    Returns:
        dict: Update result with changes made
    """
    changes = {
        "type": graph_type,
        "sql_query": sql_query
    }
    if title:
        changes["title"] = title
        
    return {
        "status": "success",
        "timestamp": datetime.now().isoformat(),
        "updated_graph_id": graph_id,
        "changes": changes,
        "message": f"Successfully updated graph {graph_id}"
    }

# Model configuration
model = LiteLLMModel(
    client_args={
        "api_key": "<KEY>",
    },
    # **model_config
    model_id="gemini/gemini-2.5-flash",
    params={
        "max_tokens": 1000,
        "temperature": 0.7,
    }
)

# Agent configuration
agent = Agent(model=model, tools=[
    get_recent_financial_data,
    get_current_graphs, 
    analyze_and_update_graphs,
    find_interesting_relations,
    determine_graph_config,
    find_least_useful_graph,
    remove_graph,
    add_new_graph,
    update_graph,
    calculator
])

def get_enhanced_financial_prompt(max_graphs: int) -> str:
    """
    Returns the comprehensive financial analysis prompt.
    
    Args:
        max_graphs (int): Maximum number of graphs to maintain
        
    Returns:
        str: Detailed financial analysis prompt
    """
    return f"""
You are a sophisticated Financial Data Analysis Agent responsible for maintaining an optimal dashboard of financial insights. Execute the complete graph management workflow with deep analytical rigor:

## 1. CURRENT GRAPH ANALYSIS & OPTIMIZATION

**Step 1a: Get Current State**
- Use `get_current_graphs()` to retrieve all existing graphs
- Catalog each graph's: id, type, title, sql_query, and current relevance

**Step 1b: Analyze & Update Existing Graphs** 
- Use `analyze_and_update_graphs(current_graphs)` to evaluate each graph for:
  - **Data Freshness**: Does the query capture the most relevant time periods?
  - **Graph Type Optimization**: Is the current chart type optimal for the data pattern?
  - **Query Efficiency**: Can the SQL be optimized for better insights?
  - **Visual Clarity**: Does the title accurately represent the insight?
- Update graphs using `update_graph(graph_id, new_type, new_sql, new_title)` if improvements are identified

## 2. COMPREHENSIVE FINANCIAL DATA ANALYSIS

**Step 2a: Data Acquisition**
- Use `get_recent_financial_data(limit=100)` to fetch the latest financial records
- Ensure data coverage includes recent trends and sufficient historical context

**Step 2b: Advanced Financial Relationship Discovery**
- Use `find_interesting_relations(financial_data)` to perform sophisticated analysis including:

### FRAUD & RISK ANALYSIS
- **Fraud Detection Patterns**: Transaction amount anomalies, frequency spikes, timing irregularities
- **Risk Concentration**: Geographic, demographic, or temporal risk clustering
- **Anomaly Detection**: Unusual payment methods, transaction sizes, customer behaviors

### PERFORMANCE METRICS  
- **Profitability Analysis**: Revenue trends, margin analysis, cost-efficiency ratios
- **Liquidity Assessment**: Cash flow patterns, working capital trends
- **Operational Efficiency**: Processing times, approval rates, conversion metrics

### CUSTOMER & MARKET INSIGHTS
- **Customer Segmentation**: Lifetime value analysis, acquisition cost trends, retention patterns
- **Geographic Performance**: Regional revenue distribution, market penetration analysis
- **Seasonal Intelligence**: Cyclical patterns, holiday impacts, quarterly variations

### CORRELATION ANALYSIS
- **Multi-variable Relationships**: How financial metrics interconnect and influence each other
- **Leading Indicators**: Early warning signals that predict future performance
- **Risk Factor Combinations**: What metric combinations increase fraud or default probability

You are not allowed to use more data than 100 rows

## 3. INTELLIGENT GRAPH CREATION

**Step 3a: Relationship Prioritization**
For each discovered relation, evaluate:
- **Business Impact**: How significant for decision-making?
- **Trend Strength**: How clear and consistent is the pattern?
- **Actionability**: Can stakeholders act on this insight?
- **Novelty**: Does this reveal new information?

**Step 3b: Optimal Graph Configuration**
- Use `determine_graph_config(relation)` for high-priority relationships:
  - **Graph Type Selection**:
    - `bar`: Categorical comparisons (fraud rates by region, revenue by segment)
    - `line`: Time series trends (revenue over time, customer growth)
    - `scatter`: Correlation analysis (risk vs return, cost vs volume)
    - `pie`: Composition analysis (transaction types, market share)
  - **SQL Query Crafting**: Create queries that:
    - Include relevant time filters (last 30/90/365 days)
    - Apply appropriate aggregations (SUM, AVG, COUNT, percentiles)
    - Include meaningful groupings and proper sorting
    - Filter noise and focus on statistically significant data
  - **Title Generation**: Create descriptive, actionable titles like:
    - "Q4 2024 Fraud Rate Surge: Geographic Risk Analysis"
    - "Customer Acquisition Cost vs Lifetime Value by Market Segment"
    - "Processing Time Impact on Transaction Success Rates"
    - "Monthly Revenue Waterfall: Q3 to Q4 2024 Bridge Analysis"
    - "Customer Profitability Matrix: Acquisition Cost vs Lifetime Value"

## 4. STRATEGIC GRAPH PORTFOLIO MANAGEMENT

**Step 4a: Portfolio Assessment**  
- Evaluate current graph count against maximum limit ({max_graphs})
- If at capacity, use `find_least_useful_graph(current_graphs)` based on:
  - **Data Staleness**: Graphs showing outdated or irrelevant trends
  - **Low Information Value**: Visualizations that don't drive decisions
  - **Redundancy**: Similar insights already captured elsewhere
  - **Statistical Insignificance**: Patterns that have become noise

**Step 4b: Strategic Removal & Addition**
- If removal needed: Use `remove_graph(graph_id)` for the least valuable visualization
- Use `add_new_graph(graph_config)` to introduce the most impactful new insight
- Show me the add_new_graph input
- Ensure final portfolio provides:
  - Balanced coverage across fraud, performance, customer, and market domains
  - Mix of time horizons (real-time alerts, short-term trends, long-term patterns)  
  - Actionable intelligence for different stakeholder groups

## 5. COMPREHENSIVE REPORTING

**Step 5a: Executive Summary**
Document:
- **Actions Taken**: Graphs updated, removed, and added with counts
- **Key Insights Discovered**: Most significant financial patterns identified
- **Data Quality Assessment**: Any anomalies, gaps, or data issues observed  
- **Trend Analysis**: Emerging patterns requiring continued monitoring
- **Business Recommendations**: Suggested follow-up analysis or actions

**Step 5b: Technical Details**
- List specific SQL queries modified or created
- Document graph type changes with rationale
- Report on data volume and analysis time period
- Note any performance issues or tool limitations

## SUCCESS CRITERIA
- Maintain exactly {max_graphs} high-value, unique visualizations
- Each graph provides actionable financial intelligence
- Portfolio covers key domains: fraud detection, performance monitoring, customer insights, market analysis
- All graphs use optimal visualization types for their data patterns
- SQL queries are efficient and capture meaningful time periods
- Insights are current, relevant, and statistically significant
- Analysis drives business decisions and risk management

Execute this workflow systematically, ensuring each step builds comprehensive financial intelligence for strategic decision-making.
"""

def run_graph_management_agent(interval_seconds=20, max_iterations=None, max_graphs=6):
    """
    Runs the enhanced financial graph management agent continuously.
    
    Args:
        interval_seconds (int): How often to run the agent (default: 20 seconds)
        max_iterations (int): Maximum number of iterations (None for infinite)
        max_graphs (int): Maximum number of graphs to maintain (default: 6)
    """
    print(f"🚀 Starting Enhanced Financial Data Analysis Agent...")
    print(f"📊 Update interval: {interval_seconds} seconds")
    print(f"📈 Max graphs: {max_graphs}")
    print(f"🔄 Max iterations: {'Unlimited' if max_iterations is None else max_iterations}")
    print(f"⏰ Start time: {datetime.now()}")
    print("=" * 80)
    
    iteration = 0
    
    try:
        while max_iterations is None or iteration < max_iterations:
            iteration += 1
            
            print(f"\n{'='*15} 📊 FINANCIAL ANALYSIS CYCLE {iteration} {'='*15}")
            print(f"🕐 Starting comprehensive analysis at {datetime.now()}")
            
            # Use the enhanced prompt
            message = get_enhanced_financial_prompt(max_graphs)
            
            try:
                result = agent(message)
                print(f"✅ [CYCLE {iteration}] Financial analysis completed successfully")
                
            except Exception as e:
                print(f"❌ [CYCLE {iteration}] Agent error: {e}")
            
            if max_iterations is None or iteration < max_iterations:
                print(f"⏳ [CYCLE {iteration}] Waiting {interval_seconds} seconds until next analysis...")
                print("-" * 80)
                time.sleep(interval_seconds)
    
    except KeyboardInterrupt:
        print(f"\n\n🛑 Received interrupt signal. Stopping after {iteration} iterations.")
        print(f"🏁 End time: {datetime.now()}")
    except Exception as e:
        print(f"\n\n💥 Unexpected error in analysis loop: {e}")
        print(f"🏁 Stopped after {iteration} iterations at {datetime.now()}")

async def run_graph_management_agent_async(interval_seconds=20, max_iterations=1, max_graphs=6):
    """
    Async version of the enhanced financial graph management agent.
    
    Args:
        interval_seconds (int): How often to run the agent
        max_iterations (int): Maximum number of iterations  
        max_graphs (int): Maximum number of graphs to maintain
    """
    print(f"🚀 Starting Async Enhanced Financial Analysis Agent...")
    print(f"📊 Update interval: {interval_seconds} seconds")
    print(f"📈 Max graphs: {max_graphs}")
    print("=" * 80)
    
    iteration = 0
    
    try:
        while max_iterations is None or iteration < max_iterations:
            iteration += 1
            
            print(f"\n🔄 [ASYNC CYCLE {iteration}] Starting comprehensive financial analysis...")
            
            # Use the enhanced prompt
            message = get_enhanced_financial_prompt(max_graphs)
            
            try:
                result = agent(message)
                print(f"✅ [ASYNC CYCLE {iteration}] Financial analysis completed")
                
            except Exception as e:
                print(f"❌ [ASYNC CYCLE {iteration}] Agent error: {e}")
            
            if max_iterations is None or iteration < max_iterations:
                agent.reset()
                await asyncio.sleep(interval_seconds)
    
    except Exception as e:
        print(f"\n💥 Async loop error: {e}")

if __name__ == "__main__":
    print("🎯 Running Enhanced Financial Analysis Agent for 10 iterations (200 seconds total)...")
    run_graph_management_agent(interval_seconds=20, max_iterations=10, max_graphs=6)

ModuleNotFoundError: No module named 'strands.models.google'